# SEIRX model runs for only TTI

In [1]:
import pandas as pd
from os.path import join
import json

# parallelisation functionality
from multiprocess import Pool
import psutil
from tqdm import tqdm

# custom functionality
from scseirx.model_nursing_home import SEIRX_nursing_home # agent-based simulation
import data_creation_functions as dcf

In [2]:
# paths for I/O
contact_network_src = '../data/contact_networks'

## Simulation function

In [3]:
def run(param_list):
    '''
    Runs an ensemble of simulations and collects observable statistics. To be 
    run in parallel on many workers. Note: I/O paths and the number of runs per 
    ensemble hare hard coded here, because I only want to pass the parameter 
    values that are being screened in the simulation run to the function via the
    parallel processing interface.
    
    Parameters:
    -----------
    param_list : iterable
        Iterable that contains the values for the parameters test_type, 
        index_case, e_screen_range and r_screen_range that are passed to the
        simulation.
    N_runs : integer
        Number of runs per ensemble.
        
    Returns:
    --------
    row : dictionary
        Dictionary of the ensemble statistics of the observables.
    '''    

    # extract the simulation parameters from the parameter list
    index_case, N_runs = param_list

    # run the ensemble with the given simulation parameters 
    ensemble_results = dcf.run_ensemble(N_runs, measures, simulation_params,
                contact_network_src, ensmbl_dst, index_case=index_case)

    # calculate the ensemble statistics
    row = dcf.evaluate_ensemble(ensemble_results, index_case)

    return row

## Screening parameters

In [4]:
# specifies, whether the index case will be introduced via an employee or a 
# resident
index_cases = ['employee', 'resident']

# ensemble_size
N_runs = 5000

screening_params = [(i, N_runs) for i in index_cases ]

print('there are {} parameter combinations'.format(len(screening_params)))

there are 2 parameter combinations


## B.1.1.7

### Simulation parameters and measures in place

In [5]:
with open('params/TTI_UK_variant_simulation_parameters.json') as json_file:
    simulation_params = json.load(json_file)
    
with open('params/TTI_measures.json') as json_file:
    measures = json.load(json_file)


### Simulation runs

In [7]:
%%time
ensmbl_dst = '../data/simulation_results/ensembles/TTI/UK_variant'

number_of_cores = psutil.cpu_count(logical=True) - 2
pool = Pool(number_of_cores)

rows = []
for row in tqdm(pool.imap_unordered(func=run, iterable=screening_params),
                total=len(screening_params)):
        rows.append(row)

# turn off your parallel workers 
pool.close()

# format and dump the results to disk
dst = '../data/simulation_results'
results = pd.DataFrame()
for row in rows:
    results = results.append(row, ignore_index=True)
    
results.reset_index()
index_cols = ['index_case']
other_cols = [c for c in results.columns if c not in index_cols]
results = results[index_cols + other_cols]

results.to_csv(join(dst,'simulations_TTI_UK_variant_{}.csv'\
                   .format(N_runs)), index=False)
results.head(3)

100%|██████████| 2/2 [11:37<00:00, 348.58s/it]

CPU times: user 122 ms, sys: 92.9 ms, total: 215 ms
Wall time: 11min 37s


,index_case,N_agents_0.025,N_agents_0.10,N_agents_0.25,N_agents_0.75,N_agents_0.90,N_agents_0.975,N_agents_mean,N_agents_median,N_agents_std,...,transmissions_std,undetected_infections_0.025,undetected_infections_0.10,undetected_infections_0.25,undetected_infections_0.75,undetected_infections_0.90,undetected_infections_0.975,undetected_infections_mean,undetected_infections_median,undetected_infections_std
0,resident,53.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,0.0,...,18.084148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,employee,53.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,0.0,...,17.793485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Wild type

In [8]:
ensmbl_dst = '../data/simulation_results/ensembles/TTI/wild_type'

### Simulation parameters and measures in place

In [9]:
with open('params/TTI_simulation_parameters.json') as json_file:
    simulation_params = json.load(json_file)
    
with open('params/TTI_measures.json') as json_file:
    measures = json.load(json_file)


### Simulation runs

In [10]:
%%time
number_of_cores = psutil.cpu_count(logical=True) - 2
pool = Pool(number_of_cores)

rows = []
for row in tqdm(pool.imap_unordered(func=run, iterable=screening_params),
                total=len(screening_params)):
        rows.append(row)

# turn off your parallel workers 
pool.close()

# format and dump the results to disk
dst = '../data/simulation_results'
results = pd.DataFrame()
for row in rows:
    results = results.append(row, ignore_index=True)
    
results.reset_index()
index_cols = ['index_case']
other_cols = [c for c in results.columns if c not in index_cols]
results = results[index_cols + other_cols]

results.to_csv(join(dst,'simulations_TTI_{}.csv'\
                   .format(N_runs)), index=False)
results.head(3)

100%|██████████| 2/2 [07:11<00:00, 215.96s/it]

CPU times: user 98.9 ms, sys: 109 ms, total: 208 ms
Wall time: 7min 12s


,index_case,N_agents_0.025,N_agents_0.10,N_agents_0.25,N_agents_0.75,N_agents_0.90,N_agents_0.975,N_agents_mean,N_agents_median,N_agents_std,...,transmissions_std,undetected_infections_0.025,undetected_infections_0.10,undetected_infections_0.25,undetected_infections_0.75,undetected_infections_0.90,undetected_infections_0.975,undetected_infections_mean,undetected_infections_median,undetected_infections_std
0,resident,53.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,0.0,...,11.030236,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,employee,53.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,0.0,...,11.212498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## B.1.1.7 & employee masks

### Simulation parameters and measures in place

In [11]:
with open('params/TTI_UK_variant_simulation_parameters.json') as json_file:
    simulation_params = json.load(json_file)
    
with open('params/TTI_masks_measures.json') as json_file:
    measures = json.load(json_file)


### Simulation runs

In [12]:
%%time
ensmbl_dst = '../data/simulation_results/ensembles/TTI/masks'

number_of_cores = psutil.cpu_count(logical=True) - 2
pool = Pool(number_of_cores)

rows = []
for row in tqdm(pool.imap_unordered(func=run, iterable=screening_params),
                total=len(screening_params)):
        rows.append(row)

# turn off your parallel workers 
pool.close()

# format and dump the results to disk
dst = '../data/simulation_results'
results = pd.DataFrame()
for row in rows:
    results = results.append(row, ignore_index=True)
    
results.reset_index()
index_cols = ['index_case']
other_cols = [c for c in results.columns if c not in index_cols]
results = results[index_cols + other_cols]

results.to_csv(join(dst,'simulations_TTI_masks_{}.csv'\
                   .format(N_runs)), index=False)
results.head(3)

100%|██████████| 2/2 [07:51<00:00, 235.84s/it]

CPU times: user 85.4 ms, sys: 106 ms, total: 192 ms
Wall time: 7min 51s


,index_case,N_agents_0.025,N_agents_0.10,N_agents_0.25,N_agents_0.75,N_agents_0.90,N_agents_0.975,N_agents_mean,N_agents_median,N_agents_std,...,transmissions_std,undetected_infections_0.025,undetected_infections_0.10,undetected_infections_0.25,undetected_infections_0.75,undetected_infections_0.90,undetected_infections_0.975,undetected_infections_mean,undetected_infections_median,undetected_infections_std
0,employee,53.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,0.0,...,11.973016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,resident,53.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,0.0,...,13.043875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
